In [27]:
from jobspy import scrape_jobs
import pandas as pd
from datetime import datetime

In [48]:
# Cell 2: California Scrape
print("Starting California scrape...")
print(f"Time: {datetime.now()}")

ca_jobs = scrape_jobs(
    site_name=["indeed"],
    search_term="data analyst",
    location="California",
    results_wanted=200,
    country_indeed='USA',
    enforce_annual_salary=True,
    description_format="html",
    verbose=1
)

ca_jobs['state'] = 'California'
ca_jobs.to_csv('../data/california_jobs.csv', index=False)
print(f"\n California: {len(ca_jobs)} jobs scraped")
print(f"Jobs by site:\n{ca_jobs['site'].value_counts()}")
print(f"Completed at: {datetime.now()}")
print("-" * 60)

Starting California scrape...
Time: 2025-11-17 00:56:31.444632

 California: 200 jobs scraped
Jobs by site:
site
indeed    200
Name: count, dtype: int64
Completed at: 2025-11-17 00:56:33.297978
------------------------------------------------------------


In [ ]:
# Cell 3: New York Scrape (Indeed Only)
print("Starting New York scrape...")
print(f"Time: {datetime.now()}")

ny_jobs = scrape_jobs(
    site_name=["indeed"],
    search_term="data analyst",
    location="New York",
    results_wanted=300,
    country_indeed='USA',
    enforce_annual_salary=True,
    description_format="html",
    verbose=1
)

ny_jobs['state'] = 'New York'
ny_jobs.to_csv('../data/newyork_jobs.csv', index=False)
print(f"\n New York: {len(ny_jobs)} jobs scraped")
print(f"Completed at: {datetime.now()}")
print("-" * 60)

Starting New York scrape...
Time: 2025-11-17 00:57:05.219843

✅ New York: 300 jobs scraped
Completed at: 2025-11-17 00:57:07.458177
------------------------------------------------------------


In [50]:
# Cell 4: Texas Scrape (Indeed Only)
print("Starting Texas scrape...")
print(f"Time: {datetime.now()}")

tx_jobs = scrape_jobs(
    site_name=["indeed"],
    search_term="data analyst",
    location="Texas",
    results_wanted=300,
    country_indeed='USA',
    enforce_annual_salary=True,
    description_format="html",
    verbose=1
)

tx_jobs['state'] = 'Texas'
tx_jobs.to_csv('../data/texas_jobs.csv', index=False)
print(f"\n Texas: {len(tx_jobs)} jobs scraped")
print(f"Completed at: {datetime.now()}")
print("-" * 60)

Starting Texas scrape...
Time: 2025-11-17 00:57:16.806693

 Texas: 300 jobs scraped
Completed at: 2025-11-17 00:57:19.187268
------------------------------------------------------------


In [52]:
# Cell 5: Combine All Data
print("Combining all state data...")

# Read all CSVs
ca_jobs = pd.read_csv('../data/california_jobs.csv')
ny_jobs = pd.read_csv('../data/newyork_jobs.csv')
tx_jobs = pd.read_csv('../data/texas_jobs.csv')

# Combine
all_jobs = pd.concat([ca_jobs, ny_jobs, tx_jobs], ignore_index=True)

# Save combined CSV
all_jobs.to_csv('../data/all_states_jobs.csv', index=False)

print(f"\n Combined data saved!")
print(f"Total jobs: {len(all_jobs)}")
print(f"  - California: {len(ca_jobs)}")
print(f"  - New York: {len(ny_jobs)}")
print(f"  - Texas: {len(tx_jobs)}")
print("\n" + "=" * 60)

Combining all state data...

 Combined data saved!
Total jobs: 800
  - California: 200
  - New York: 300
  - Texas: 300



In [53]:
# Cell 7: Create Excel File with Separate Sheets
print("Creating Excel file with separate sheets...")

# Key columns to keep
columns_to_keep = [
    'title', 
    'company', 
    'location', 
    'min_amount', 
    'max_amount', 
    'currency',
    'interval',
    'salary_source',
    'date_posted', 
    'job_type', 
    'is_remote', 
    'job_url',
    'description'
]

# Install openpyxl if needed
try:
    import openpyxl
except ImportError:
    import subprocess
    import sys
    print("Installing openpyxl...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "openpyxl", "--break-system-packages"])
    print(" openpyxl installed!")

# Create Excel writer
with pd.ExcelWriter('../data/jobs_by_state.xlsx', engine='openpyxl') as writer:
    # Write each state to a separate sheet
    ca_jobs[columns_to_keep].to_excel(writer, sheet_name='California', index=False)
    ny_jobs[columns_to_keep].to_excel(writer, sheet_name='New York', index=False)
    tx_jobs[columns_to_keep].to_excel(writer, sheet_name='Texas', index=False)
    
    # Add combined sheet with state column
    all_with_state = all_jobs[columns_to_keep + ['state']].copy()
    all_with_state.to_excel(writer, sheet_name='All States', index=False)

print("\n Created Excel file: jobs_by_state.xlsx")
print("   Sheets:")
print("   - California: {} jobs".format(len(ca_jobs)))
print("   - New York: {} jobs".format(len(ny_jobs)))
print("   - Texas: {} jobs".format(len(tx_jobs)))
print("   - All States: {} jobs".format(len(all_jobs)))
print("\n All done! Check your /data folder for the files.")

Creating Excel file with separate sheets...

 Created Excel file: jobs_by_state.xlsx
   Sheets:
   - California: 200 jobs
   - New York: 300 jobs
   - Texas: 300 jobs
   - All States: 800 jobs

 All done! Check your /data folder for the files.


In [54]:
# Cell 8: Create Summary Report
print("Creating summary report...")

# Calculate summary statistics
summary = {
    'State': ['California', 'New York', 'Texas', 'TOTAL'],
    'Total Jobs': [
        len(ca_jobs), 
        len(ny_jobs), 
        len(tx_jobs), 
        len(all_jobs)
    ],
    'Jobs with Salary': [
        ca_jobs['min_amount'].notna().sum(),
        ny_jobs['min_amount'].notna().sum(),
        tx_jobs['min_amount'].notna().sum(),
        all_jobs['min_amount'].notna().sum()
    ],
    'Avg Min Salary': [
        ca_jobs['min_amount'].mean(),
        ny_jobs['min_amount'].mean(),
        tx_jobs['min_amount'].mean(),
        all_jobs['min_amount'].mean()
    ],
    'Avg Max Salary': [
        ca_jobs['max_amount'].mean(),
        ny_jobs['max_amount'].mean(),
        tx_jobs['max_amount'].mean(),
        all_jobs['max_amount'].mean()
    ],
    'Remote Jobs': [
        ca_jobs['is_remote'].sum(),
        ny_jobs['is_remote'].sum(),
        tx_jobs['is_remote'].sum(),
        all_jobs['is_remote'].sum()
    ]
}

summary_df = pd.DataFrame(summary)

# Format currency columns
for col in ['Avg Min Salary', 'Avg Max Salary']:
    summary_df[col] = summary_df[col].apply(
        lambda x: f"${x:,.0f}" if pd.notna(x) else "N/A"
    )

print("\n" + "=" * 80)
print("JOB MARKET SUMMARY - Data Analyst Positions")
print("=" * 80)
print(summary_df.to_string(index=False))
print("=" * 80)

# Save summary
summary_df.to_csv('../data/summary_report.csv', index=False)
print("\n Summary saved to: summary_report.csv")

Creating summary report...

JOB MARKET SUMMARY - Data Analyst Positions
     State  Total Jobs  Jobs with Salary Avg Min Salary Avg Max Salary  Remote Jobs
California         200               173        $94,885       $130,866           45
  New York         300               252        $90,240       $121,277           72
     Texas         300               143        $90,503       $123,795           59
     TOTAL         800               568        $91,721       $124,832          176

 Summary saved to: summary_report.csv
